# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [14]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

### Streaming tick data

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2019, 4, 15, 17, 27, 47, 50672, tzinfo=datetime.timezone.utc), bid=1.13035, bidSize=2000000, ask=1.13036, askSize=2000000, prevBid=1.13034, prevBidSize=3000000, prevAsk=1.13035, prevAskSize=1000000, high=1.13205, low=1.1298, close=1.1302, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

The price of Forex ticks is always nan. To get a midpoint price use ``midpoint()`` or ``marketPrice()``.

The tickers are kept live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.130355

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,1500000,1.13033,1.13034,2000000,1.13205,1.1298,1.1302
USDJPY,2000000,112.004,112.005,3000000,112.1,111.885,112.02
GBPUSD,1000000,1.30989,1.30993,2000000,1.31195,1.307,1.3078
USDCHF,3000000,1.0042,1.00425,1000000,1.00425,1.0009,1.0025
USDCAD,1000000,1.33721,1.33724,1000000,1.33905,1.32975,1.3325
AUDUSD,5000000,0.71764,0.71767,1000000,0.71825,0.71635,0.71725


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

### Tick by Tick data ###

The ticks in the previous section are time-sampled by IB in order to cut on bandwidth. So with ``reqMktdData`` not every tick from the exchanges is sent. The promise of ``reqTickByTickData`` is to send every tick, just how it appears in the TWS Time & Sales window. This functionality is severly nerfed by a total of just three simultaneous subscriptions, where bid-ask ticks and sale ticks also use up a subscription each.

The tick-by-tick updates are available from ``ticker.tickByTicks`` and are signalled by ``ib.pendingTickersEvent`` or ``ticker.updateEvent``.

In [8]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2019, 4, 15, 17, 28, 19, 104035, tzinfo=datetime.timezone.utc), bid=1.13034, bidSize=1000000, ask=1.13035, askSize=1000000, prevBid=1.13033, prevBidSize=1500000, prevAsk=1.13034, prevAskSize=2000000, high=1.13205, low=1.1298, close=1.1302, ticks=[], tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2019, 4, 15, 17, 28, 19, 104035, tzinfo=datetime.timezone.utc), bidPrice=1.13034, askPrice=1.13035, bidSize=1000000, askSize=1000000, tickAttribBidAsk=TickAttribBidAsk())], domBids=[], domAsks=[], domTicks=[])


### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [9]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2019, 4, 15, 17, 28, 18, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=1.13034, priceAsk=1.13035, sizeBid=1000000, sizeAsk=1000000)

In [10]:
ib.disconnect()